In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, util
from skimage.transform import warp_polar

ModuleNotFoundError: No module named 'skimage'

In [ ]:
# Display helper for consistent figure sizing
def show_images(images, titles=None, cmap='gray', figsize=(14,6)):
    n = len(images)
    plt.figure(figsize=figsize)
    for i, img in enumerate(images):
        ax = plt.subplot(1, n, i+1)
        ax.imshow(img, cmap=cmap, origin='lower')
        ax.axis('off')
        if titles:
            ax.set_title(titles[i])
    plt.tight_layout()
    plt.show()

# Compute log-magnitude spectrum for visualization
def log_magnitude_spectrum(img):
    F = np.fft.fftshift(np.fft.fft2(img))
    mag = np.abs(F)
    return np.log1p(mag)

In [ ]:

folder = "/coil-20-unproc/coil-20-unprocs"   # <-- change to your folder
if 'image_path' not in globals():
    exts = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp')
    image_path = None
    if os.path.isdir(folder):
        for fname in os.listdir(folder):
            if fname.lower().endswith(exts):
                image_path = os.path.join(folder, fname)
                break
    if image_path is None:
        raise FileNotFoundError(f"No image found in {folder}. Set image_path manually.")

print("Loading:", image_path)
img_color = io.imread(image_path)
# Convert to grayscale float [0,1]
if img_color.ndim == 3:
    img_gray = color.rgb2gray(img_color)
else:
    img_gray = util.img_as_float(img_color)

show_images([img_gray], titles=["Original (grayscale)"], figsize=(6,6))

In [ ]:
# Cell 3 — Polar transform (linear)
# Choose center (defaults to image center)
h, w = img_gray.shape[:2]
center = (w//2, h//2)

# radius: max radius to fit inside image from center
max_radius = int(np.hypot(max(center[0], w - center[0]), max(center[1], h - center[1])))

# output_shape: (radius_samples, angle_samples)
radius_samples = max_radius
angle_samples = 720  # increase for smoother angular resolution

polar_img = warp_polar(img_gray, center=center, radius=max_radius,
                       output_shape=(radius_samples, angle_samples),
                       scaling='linear')  # 'linear' polar

# visualize
show_images([polar_img], titles=["Polar transform (linear)"], figsize=(10,5))


In [ ]:
# Cell 4 — Log-polar transform (radius mapped logarithmically)
# warp_polar supports scaling='log' in recent skimage versions
try:
    logpolar_img = warp_polar(img_gray, center=center, radius=max_radius,
                              output_shape=(radius_samples, angle_samples),
                              scaling='log')
    used_scaling = 'built-in log'
except Exception as e:
    # fallback: create manual log-polar by remapping radius axis
    used_scaling = 'manual log'
    # Create radius/angle grids
    angles = np.linspace(0, 2*np.pi, angle_samples, endpoint=False)
    log_r = np.linspace(0, np.log(max_radius+1), radius_samples)
    R = np.exp(log_r) - 1  # from log space back to radius
    # build sampling grid
    coords = []
    for r in R:
        xs = center[0] + r * np.cos(angles)
        ys = center[1] + r * np.sin(angles)
        coords.append(np.vstack([ys, xs]))  # (2, angle_samples)
    coords = np.stack(coords, axis=0)  # (radius_samples, 2, angle_samples)
    # bilinear interpolation using map_coordinates
    from scipy.ndimage import map_coordinates
    coords_flat = np.vstack([coords[:,:,i].T for i in range(angle_samples)])  # large stack
    # easier: sample per-angle
    logpolar_img = np.zeros((radius_samples, angle_samples), dtype=img_gray.dtype)
    for j in range(angle_samples):
        ys = coords[:,0,j]
        xs = coords[:,1,j]
        logpolar_img[:, j] = map_coordinates(img_gray, [ys, xs], order=1, mode='reflect')

print("Log-polar transform method:", used_scaling)
show_images([logpolar_img], titles=["Log-polar transform"], figsize=(10,5))
